In [8]:
import os
from dotenv import load_dotenv
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

In [9]:
load_dotenv()
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))

DATA_DIR = os.path.join(BASE_DIR, "data")
CHROMA_DIR = os.path.join(BASE_DIR, "chroma_db")

In [10]:
os.makedirs(DATA_DIR, exist_ok=True)

# Sample KB files (name -> content)
sample_docs = {

    "Account_guide.txt":"""How to change your plan
You can change your Netflix plan by following these steps:

Go to the Change Plan page in a browser. You may need to sign in if you haven't already.

Choose the plan you'd like, then select Continue or Update.

If you don't have the option to change your plan, see "Can't change plan" below.

Tap or click Confirm Change or Confirm.

Changing to a higher-priced plan takes effect immediately so you can enjoy all of the added features. Your billing date may change based on the remaining balance of your last payment, or you may be charged a prorated amount for the balance.

Changing to a lower-priced plan takes effect on your next billing date. You can continue to use the features of the higher-priced plan until then.

Note: If you have Netflix through a partner package or package add-on, search our Help Center for the article on your partner to find details on plan change timing and eligibility.""",
    
    "Cant_change_plan.txt":"""Can't change plan

If you aren’t able to change your plan, it could be for a number of reasons:

Account is on hold

Account is paused

A Kids profile is active whe"n you go to the Change Plan page. See How to switch profiles to change to a different one.

You changed to a lower-priced plan this billing period. (You'll need to wait until the next billing period to make another plan change.)

Note: If you pay for Netflix through a partner or get Netflix as part of a package you may not have the option to change your plan.""",

    "in_use.txt":"""Netflix says account is already in use
If you get the error code E123 or any of these messages when you play a TV show or movie:

Too many people are using your account right now.

Your Netflix account is in use on another device.

Your Netflix account is in use on too many devices.

It means the maximum number of devices allowed to watch at the same time using your Netflix account has been reached. Your Netflix plan determines how many devices are allowed to watch at the same time using your account.

To fix the problem:

On a different device using your account, stop playing Netflix or close the Netflix app. If you don't know which devices on your account might be playing Netflix at the same time, you can check which devices recently watched.

Note:After stopping Netflix on a different device, you may need to wait 5-10 minutes before you can watch on your device.

Or, upgrade your Netflix plan to allow more devices to watch at the same time (up to 4 with the Premium plan).

If your Netflix account is being used without permission, you can stop someone from using your account.""",

    "error_code.txt":"""Netflix Error NW-2-5
You might see this message on your TV or TV streaming device when you try to open Netflix:

Netflix has encountered an error. Retrying in [X] seconds.
Code: NW-2-5

Error code NW-2-5 means your device took too long to connect to Netflix. This usually points to an internet connection problem on your device or home network.

Follow the steps below to fix the issue. If it isn't fixed after doing these steps, you'll need to contact your internet service provider (ISP).


1. Check your internet connection


2. Restart your device


3. Restart your home network


4. Move your Wi-Fi router


5. Set DNS to automatic (PlayStation/Xbox only)

Netflix Error ui-800-3
If the error code shown on your device also has numbers in parentheses ( ), search for that exact error code in our Help Center to get the right steps. Example: ui-800-3 (123456)

If the error code on your device shows only ui-800-3, it means information stored on your device needs to be refreshed, or a network issue stopped Netflix from opening.

To fix the problem, follow the steps for your device.

TV or TV-connected device

These steps can fix this error on your TV or a device that connects to your TV, like a streaming stick, media player, set-top box, or Blu-ray player.

Restart your device

Restart your home network

Sign out of Netflix

Restore your default connection settings.
Netflix Error tvq-pb-101
Check the error code on your TV. If there are additional numbers or letters inside parentheses ( ), search for that exact error code in our Help Center. There might be an article with steps that fix your exact error code.

This error happens when an issue with data stored on your device stops Netflix from playing.

To fix the problem, follow the steps for your device.


Smart TV


Restart your device


Sign out of Netflix

If you use a Vizio Smart TV and get this issue after following the steps above:


Contact the device manufacturer.""",

    "watching.txt":"""Fix a problem on your Android phone or tablet
If Netflix isn't working on your Android phone or tablet, you can fix common issues with these steps.


Turn your device off, then back on


Clear the Netflix app data


Reinstall the Netflix app


Update your Android OS version


Test your internet connection


Restart your home network.""",

    "compatible.txt":"""Netflix says, 'This app is not compatible with your device.'
If you're getting the error message below when trying to use or install Netflix on your Android device, it usually means that your device needs to be restarted or that it isn't Play Protect certified. (That means it won't be able to get the Netflix app from the Google Play Store.)

Error message

This app is not compatible with your device.

To fix the problem:


Restart your device


Check your device's Play Protect status."""

    
}
    

# Write files
for filename, content in sample_docs.items():
    file_path = os.path.join(DATA_DIR, filename)
    with open(file_path, "w", encoding="utf-8") as f:
        f.write(content)

print(f"Created {len(sample_docs)} sample KB documents in {DATA_DIR}")

Created 6 sample KB documents in C:\Users\bhara\Documents\help_bot\data


In [4]:
CHROMA_DIR = os.path.join(BASE_DIR, "chroma_db")
os.makedirs(CHROMA_DIR, exist_ok=True)

print(f"✅ Chroma DB directory created at: {CHROMA_DIR}")

✅ Chroma DB directory created at: C:\Users\bhara\Documents\help_bot\chroma_db


In [5]:
# -----------------------------
# LOAD DOCUMENTS
# -----------------------------
print("Loading documents...")
loader = DirectoryLoader(DATA_DIR, glob="*.txt", loader_cls=lambda path: TextLoader(path, encoding="utf-8"))
documents = loader.load()

print(f"✅ Loaded {len(documents)} documents.")

# -----------------------------
# SPLIT INTO CHUNKS
# -----------------------------
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  # max characters per chunk
    chunk_overlap=50 # overlap for better context
)
docs = text_splitter.split_documents(documents)

print(f"📝 Split into {len(docs)} chunks.")

# -----------------------------
# EMBEDDINGS (HuggingFace)
# -----------------------------
print("🔍 Creating embeddings...")
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# -----------------------------
# CREATE / UPDATE CHROMA DB
# -----------------------------
print("💾 Saving to Chroma DB...")
db = Chroma.from_documents(docs, embeddings, persist_directory=CHROMA_DIR)
db.persist()

print(f"🎉 Ingestion complete! Chroma DB is ready at: {CHROMA_DIR}")

Loading documents...
✅ Loaded 9 documents.
📝 Split into 18 chunks.
🔍 Creating embeddings...


C:\Users\bhara\AppData\Local\Temp\ipykernel_1460\494788548.py:25: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")



💾 Saving to Chroma DB...
🎉 Ingestion complete! Chroma DB is ready at: C:\Users\bhara\Documents\help_bot\chroma_db


C:\Users\bhara\AppData\Local\Temp\ipykernel_1460\494788548.py:32: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()


In [6]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

# Paths (same as ingestion)
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
CHROMA_DIR = os.path.join(BASE_DIR, "chroma_db")

# Load embeddings (must match ingestion model)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Load existing Chroma DB
db = Chroma(persist_directory=CHROMA_DIR, embedding_function=embeddings)


C:\Users\bhara\AppData\Local\Temp\ipykernel_1460\1671407650.py:12: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(persist_directory=CHROMA_DIR, embedding_function=embeddings)
